In [22]:
# Re-load necessary libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Define file path
file_path = "data.csv"

# Load the CSV with correct decimal separator and handle percentage column
df = pd.read_csv(file_path, delimiter=',', decimal=',', dtype=str)

# Convert revenue to numeric (handling European decimal format)
df['revenue'] = df['revenue'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)

# Convert year and month to integers
df['year'] = df['year'].astype(int)
df['month'] = df['month'].astype(int)

# Convert month_contr to a float (handling percentage format)
df['month_contr'] = df['month_contr'].str.replace('%', '', regex=False).str.replace(',', '.').astype(float) / 100

# --- STEP 1: ETS FORECASTING ---
# Aggregate revenue by year and month
train_data = df.groupby(['year', 'month'])['revenue'].sum()

# Ensure time index is correctly formatted
train_data.index = pd.to_datetime(train_data.index.map(lambda x: f"{x[0]}-{x[1]:02d}"))
train_data = train_data.sort_index()

# Fit the ETS model (without seasonality due to limited data)
ets_model = ExponentialSmoothing(train_data, trend='add').fit()

# Forecast next 12 months (Full Year 2025)
ets_forecast_2025 = ets_model.forecast(12)

# --- STEP 2: WEIGHTED MOVING AVERAGE (WMA) FORECASTING ---
# Get total revenue of 2024
total_revenue_2024 = df[df['year'] == 2024]['revenue'].sum()

# Calculate WMA forecast for 2025 using 2024's month contributions
# Ensure month_contr contains exactly 12 values for WMA
month_contr_2024 = df[df['year'] == 2024].sort_values(by="month")['month_contr'].values

# Ensure there are 12 values for month_contr (Jan–Dec)
if len(month_contr_2024) != 12:
    raise ValueError(f"Expected 12 month contribution values, but got {len(month_contr_2024)}")

# Calculate WMA forecast for all 12 months of 2025
wma_forecast_2025 = total_revenue_2024 * month_contr_2024

# Ensure both forecasts are 12 values long
if len(ets_forecast_2025) != 12 or len(wma_forecast_2025) != 12:
    raise ValueError("Forecast arrays are not the same length.")

# --- STEP 3: MAPE CALCULATION ---
# Extract actual revenue for 2024 (to compare accuracy)
actual_2024 = df[df['year'] == 2024]['revenue'].values

# Calculate MAPE for ETS
mape_ets = np.mean(np.abs((actual_2024 - ets_forecast_2025) / actual_2024)) * 100 if actual_2024.shape == ets_forecast_2025.shape else None

# Calculate MAPE for WMA
mape_wma = np.mean(np.abs((actual_2024 - wma_forecast_2025) / actual_2024)) * 100 if actual_2024.shape == wma_forecast_2025.shape else None

# --- STEP 4: EXPORT FORECAST TO CSV ---
# Prepare final forecast dataframe
forecast_df = pd.DataFrame({
    'Month': range(1, 13),
    'ETS_Forecast': ets_forecast_2025.values,
    'WMA_Forecast': wma_forecast_2025
})

# Save to CSV
output_file_path = "forecast_2025.csv"
forecast_df.to_csv(output_file_path, index=False)

# --- STEP 5: DISPLAY RESULTS ---
print(f"ETS MAPE: {mape_ets:.2f}%" if mape_ets is not None else "ETS MAPE could not be calculated due to shape mismatch.")
print(f"WMA MAPE: {mape_wma:.2f}%" if mape_wma is not None else "WMA MAPE could not be calculated due to shape mismatch.")
print(f"Forecast saved as {output_file_path}")

ETS MAPE: 23.95%
WMA MAPE: 0.03%
Forecast saved as forecast_2025.csv


/Users/matheus/Documents/criteo_case/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/matheus/Documents/criteo_case/.venv/lib/python3.13/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
